# Entregable 1: Implementación de Multithreading en PyQuil

**Computación Cuántica - Sesión 14**

Este entregable explora cómo ejecutar simulaciones cuánticas en paralelo usando multithreading en PyQuil, aplicando el problema del Tramposo en diferentes escenarios.

## Importación de librerías necesarias

Se importan PyQuil, utilidades de concurrencia y otras librerías para construir, simular y analizar circuitos cuánticos en paralelo.

In [ ]:
from pyquil import Program, get_qc
from pyquil.gates import H, X, CNOT, MEASURE
from pyquil.quilbase import Declare
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

## 1. Creación de circuitos cuánticos para diferentes tipos de juego

Se define una función para construir circuitos cuánticos según el tipo de juego: justo, tramposo o contraataque.

In [ ]:
def crear_circuito_juego(tipo_juego='justo', num_qubits=2):
    """
    Crea un circuito cuántico según el tipo de juego.
    - 'justo': ambos jugadores aleatorios
    - 'tramposo': Jugador 1 siempre 0, Jugador 2 aleatorio
    - 'contraataque': Jugador 1 siempre 0, Jugador 2 aleatorio, Jugador 3 siempre 1
    """
    p = Program()
    ro = p.declare('ro', 'BIT', num_qubits)
    if tipo_juego == 'justo':
        p += H(0)
        p += H(1)
    elif tipo_juego == 'tramposo':
        p += H(1)
    elif tipo_juego == 'contraataque':
        p += H(1)
        p += X(2)
    for i in range(num_qubits):
        p += MEASURE(i, ro[i])
    return p

### Explicación

- Se construye el circuito según el tipo de juego.
- Se declaran los registros clásicos para almacenar los resultados de las mediciones.
- Se aplican las puertas cuánticas correspondientes a cada jugador.

## 2. Ejecución de simulaciones en paralelo (multithreading)

Se define una función para ejecutar simulaciones cuánticas en diferentes hilos, permitiendo aprovechar múltiples núcleos de CPU y acelerar el análisis estadístico.

In [ ]:
def ejecutar_simulacion_single(qc, programa, num_shots, thread_id):
    """
    Ejecuta una simulación en un thread y retorna resultados y metadatos.
    """
    start_time = time.time()
    p_wrapped = programa.copy()
    p_wrapped.wrap_in_numshots_loop(num_shots)
    executable = qc.compile(p_wrapped)
    results = qc.run(executable)
    end_time = time.time()
    return {
        'thread_id': thread_id,
        'results': results,
        'execution_time': end_time - start_time,
        'num_shots': num_shots
    }

### Explicación

- Cada hilo ejecuta una simulación independiente del circuito cuántico.
- Se recopilan los resultados y el tiempo de ejecución de cada hilo.

## 3. Análisis de resultados de los juegos

Se define una función para analizar los resultados de las simulaciones y calcular estadísticas como victorias, empates y otros indicadores relevantes.

In [ ]:
def analizar_resultados_juego(results, num_jugadores=2):
    """
    Analiza los resultados del juego y retorna estadísticas.
    """
    num_shots = len(results)
    if num_jugadores == 2:
        j1_gana = sum(1 for r in results if r[0] == 0 and r[1] == 1)
        j2_gana = sum(1 for r in results if r[0] == 1 and r[1] == 0)
        empates = sum(1 for r in results if r[0] == r[1])
        return {
            'jugador1_gana': j1_gana,
            'jugador2_gana': j2_gana,
            'empates': empates,
            'total': num_shots
        }
    elif num_jugadores == 3:
        j1_gana = sum(1 for r in results if r[0] == 0 and r[1] == 1)
        j2_gana = sum(1 for r in results if r[0] == 1 and r[1] == 0)
        j3_siempre_cruz = all(r[2] == 1 for r in results)
        empates = sum(1 for r in results if r[0] == r[1])
        return {
            'jugador1_gana': j1_gana,
            'jugador2_gana': j2_gana,
            'jugador3_siempre_cruz': j3_siempre_cruz,
            'empates': empates,
            'total': num_shots
        }

### Explicación

- Se cuentan las victorias de cada jugador y los empates.
- Para el caso de tres jugadores, se verifica si el tercer jugador siempre obtiene cruz.

## 4. Ejecución de simulaciones en paralelo y consolidación de resultados

Se utiliza un `ThreadPoolExecutor` para lanzar múltiples simulaciones en paralelo, consolidar los resultados y analizar el rendimiento.

In [ ]:
def ejecutar_multithreading(tipo_juego='justo', num_threads=4, shots_por_thread=50):
    """
    Ejecuta múltiples simulaciones en paralelo usando multithreading y analiza los resultados.
    """
    num_qubits = 3 if tipo_juego == 'contraataque' else 2
    qc_name = f'{num_qubits}q-qvm'
    qc = get_qc(qc_name)
    programa = crear_circuito_juego(tipo_juego, num_qubits)
    resultados_threads = []
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        futures = {
            executor.submit(ejecutar_simulacion_single, qc, programa, shots_por_thread, i): i
            for i in range(num_threads)
        }
        for future in as_completed(futures):
            result = future.result()
            resultados_threads.append(result)
    todos_los_resultados = []
    for r in resultados_threads:
        todos_los_resultados.extend(r['results'])
    estadisticas = analizar_resultados_juego(np.array(todos_los_resultados), num_jugadores=num_qubits)
    return resultados_threads, estadisticas

### Explicación

- Se lanzan múltiples hilos para ejecutar simulaciones en paralelo.
- Se consolidan los resultados de todos los hilos y se analizan las estadísticas globales.

## 5. Comparación de rendimiento: secuencial vs multithreading

Se compara el tiempo de ejecución entre la versión secuencial y la versión paralela para evidenciar la mejora de rendimiento usando multithreading.

In [ ]:
def comparar_con_sin_multithreading():
    """
    Compara el rendimiento entre la ejecución secuencial y la paralela.
    """
    num_threads = 4
    shots_totales = 200
    shots_por_thread = shots_totales // num_threads
    # Secuencial
    start_seq = time.time()
    qc = get_qc('2q-qvm')
    programa = crear_circuito_juego('justo', 2)
    _ = ejecutar_simulacion_single(qc, programa, shots_totales, 0)
    tiempo_secuencial = time.time() - start_seq
    # Paralelo
    start_par = time.time()
    _, _ = ejecutar_multithreading('justo', num_threads, shots_por_thread)
    tiempo_paralelo = time.time() - start_par
    speedup = tiempo_secuencial / tiempo_paralelo
    return tiempo_secuencial, tiempo_paralelo, speedup

### Explicación

- Se mide el tiempo de ejecución de la simulación secuencial y paralela.
- Se calcula el speedup obtenido por el uso de multithreading.

## 6. Conclusiones y aplicaciones

- El multithreading permite ejecutar simulaciones cuánticas en paralelo, acelerando el análisis estadístico.
- Es útil para grandes volúmenes de simulaciones y exploración de parámetros.
- Se observa una mejora significativa en el rendimiento respecto a la ejecución secuencial.

**Aplicaciones:**
- Análisis estadístico de circuitos cuánticos
- Exploración de configuraciones y parámetros
- Simulación eficiente en hardware clásico